# Recursieve datatypes

## Voorbeeld: expressies

We hebben in het vorige hoofdstuk gezien dat we voor algemene expressies recursie nodig hebben: een argument van een operator-knoop kan weer een expressie zijn.

Dit kunnen we weergeven in de volgende definitie: (later voegen we meer operatoren toe)

In [ ]:
data Expr = Add Expr Expr
          | Mul Expr Expr
          | Val Float

Merk op:
* in de definitie van `Expr` komt `Expr` weer voor in de definiërende termen in de rechterkant: dit is *recursie*. (Letterlijk: komt weer voor)
* het alternatief `Val Float` is niet recursief. In een recursieve (data)definitie moet tenminste één alternatief niet-recursief zijn.

Met behulp van deze definitie kunnen we nu waarden construeren:

In [ ]:
expr1 = Add (Val 1) (Val 2.5)

In [ ]:
expr2 = Add (Mul (Val 2) (Val 7.5)) (Val 3)

We kunnen nu functies definiëren op `Expr`-waarden. Een dergelijke functie moet een alternatieve definitie hebben voor elk alternatief in de data-definitie. Anders gezegd: een functie die werkt op `Expr`-waarden volgt de structuur van het `Expr` data-type.

Als voorbeeld geven we een functie om een `Expr`-waarde uit te rekenen. Merk op dat we haakjes om de parameters van `eval` moeten schrijven, omdat `eval Add a b` gelezen wordt als `(eval Add) a b`

In [ ]:
eval :: Expr -> Float
eval (Add a b) = (eval a) + (eval b)
eval (Mul a b) = (eval a) * (eval b)
eval (Val a) = a

In [ ]:
eval expr1

In [ ]:
eval expr2

We schrijven hier de uitwerking van deze laatste uitdrukking uit:

```
  eval expr2
=   {def. expr2}
  eval (Add (Mul (Val 2) (Val 7.5)) (Val 3)
=   {def. eval, voor Mul-alternatief}
  (eval (Mul (Val 2) (Val 7.5)) + (eval (Val 3))
=   {def. eval, voor Add-alternatief}
  ((eval (Val 2)) * (eval (Val 7.5))) + (eval (Val 3))
=   {def. eval, voor alle Val-alternatieven}
  ((2) * (7.5)) + (3)
=   {rekenen}
  (15.0) + (3)
=   {rekenen}
  18.0

```

**Voorbeeld.** Met behulp van de functie `postfix` zetten we een `Expr`-waarde om in een string, in *postfix-formaat*: eerst de argumenten, dan de operator.

In [ ]:
postfix :: Expr -> String
postfix (Add a b) = (postfix a) ++ (postfix b) ++ " + "
postfix (Mul a b) = (postfix a) ++ (postfix b) ++ " * "
postfix (Val a) = show a ++ " "


In [ ]:
postfix expr2

**Opdracht** Maak een functie om een `Expr`-waarde om te zetten in prefix-formaat: eerst de operator, dan de argumenten.

Variant: probeer de string in Haskell-formaat te maken, waarbij je de operatoren als functies beschouwt. (Gebruik de Haskell-notatie: `(+) 3 4` - met de operatoren tussen haakjes).

## Voorbeeld: vormen

We voegen aan de grafische figuren *groepering* toe, waarbij we een lijst van grafische elementen samenvoegen tot één element. Bovendien voegen we *tranformaties* toe aan deze groepen.

Groepering is in tekenprogramma's een gebruikelijke operatie. De elementen in een groep kun je dan tegelijk verplaatsen, en bijvoorbeeld ook tegelijk kopiëren. Als je de afzonderlijke elementen wilt bewerken, kun je deze groepering (tijdelijk) opheffen.

In SVG kun je in een groep ook de vormgeving van de elementen in de groep bepalen, zoals de vulkeur of de pendikte.

(Wij voegen hier een positie toe aan een groep: dat komt overeen met een *translate* van de elementen in de groep.)

In [1]:
type Point = (Float, Float)

In [2]:
data Shape = Circle Point Float
           | Rect Point Float Float
           | Text Point String
           | Line Point Point
           | Grp Point [Shape]
             deriving (Show)

In [3]:
fig1 = Circle (10, 10) 20

In [4]:
fig4 = Line (10, 10) (100, 100)

In [5]:
attr :: String -> Float -> String
attr name value = name ++ "=\"" ++ (show value) ++ "\" "

In [6]:
tosvg :: Shape -> String
tosvg (Circle (mx, my) r) = "<circle " ++ (attr "cx" mx) ++ (attr "cy" my) ++ (attr "r" r) ++ " /> \n"
tosvg (Rect (mx, my) w h) = "<rect " ++ (attr "x" mx) ++ (attr "y" my) ++ (attr "width" w) ++ (attr "height" h) ++ "/> \n"
tosvg (Text (mx, my) s) = "<text x=" ++ (attr "x" mx) ++ (attr "y" my) ++ ">" ++ s ++ "</text> \n"
tosvg (Line (ax, ay) (bx, by)) = "<line " ++ (attr "x1" ax) ++ (attr "y1" ay) ++ (attr "x2" bx) ++ (attr "y2" by) ++ "/> \n"
tosvg (Grp (mx, my) lst) = "<g transform=\"translate(" ++ (show mx) ++ " " ++ (show my) ++ ")\" > \n" ++ elems ++ "</g>\n"
                           where svg_lst = map tosvg lst
                                 elems = foldr (++) "" svg_lst

In [7]:
putStr (tosvg fig1)

<circle cx="10.0" cy="10.0" r="20.0"  /> 


In [8]:
fig3 = Grp (10, 20) [fig1, Grp (30, 40) [fig1], fig4]

In [9]:
fig3

Grp (10.0,20.0) [Circle (10.0,10.0) 20.0,Grp (30.0,40.0) [Circle (10.0,10.0) 20.0],Line (10.0,10.0) (100.0,100.0)]


In [11]:
putStr (tosvg fig3)

<g transform="translate(10.0 20.0)" > 
<circle cx="10.0" cy="10.0" r="20.0"  /> 
<g transform="translate(30.0 40.0)" > 
<circle cx="10.0" cy="10.0" r="20.0"  /> 
</g>
<line x1="10.0" y1="10.0" x2="100.0" y2="100.0" /> 
</g>


Dit geeft de volgende figuur:

![](svg-fig3.svg) asdfsf

asdasd